In [1]:
from gurobipy import Model, GRB, quicksum
import math

In [2]:
def build_pi_set(hat_p):
    """
    hat_p: list of deviations hat_p[i]
    returns sorted list Pi = {0} ∪ {hat_p_i}
    """
    values = {0.0}
    for v in hat_p:
        values.add(float(v))
    return sorted(values)

In [3]:
def compute_durations_for_pi(pi, bar_p, hat_p):
    """
    bar_p, hat_p: lists of same length
    returns list p_pi[i] = bar_p[i] + max(hat_p[i] - pi, 0)
    """
    n = len(bar_p)
    p_pi = [0.0] * n
    for i in range(n):
        p_pi[i] = float(bar_p[i]) + max(float(hat_p[i]) - pi, 0.0)
    return p_pi

In [4]:
def solve_nominal_with_gurobi_simple(pi, bar_p, hat_p, edges, source, sink, gurobi_output=False):
    """
    bar_p, hat_p: lists (length n)
    edges: list of (i, j) with i,j in {0,...,n-1}
    source, sink: integers

    Model:
        min   sum_{(i,j)} -p_i(pi) * phi_ij
        s.t.  flow constraints

    Returns:
        total_duration (float): max project duration  = -adv_obj
        adv_obj (float): min value of sum -p_i(pi) phi_ij (≤ 0)
        phi_values: dict {(i,j): value}
    """
    p_pi = compute_durations_for_pi(pi, bar_p, hat_p)

    n = len(bar_p)
    nodes = list(range(n))

    model = Model("nominal_adversary")
    if not gurobi_output:
        model.Params.OutputFlag = 0

    # Decision variables phi_ij
    phi = {}
    for (i, j) in edges:
        phi[(i, j)] = model.addVar(vtype=GRB.BINARY, name=f"phi_{i}_{j}")

    model.update()

    # Objective: min sum -p_i(pi) * phi_ij
    obj_expr = quicksum(-p_pi[i] * phi[(i, j)] for (i, j) in edges)
    model.setObjective(obj_expr, GRB.MINIMIZE)

    # Flow constraints
    for i in nodes:
        outgoing = quicksum(phi[(u, v)] for (u, v) in edges if u == i)
        incoming = quicksum(phi[(u, v)] for (u, v) in edges if v == i)

        if i == source:
            model.addConstr(outgoing == 1.0, name=f"source_flow_{i}")
        elif i == sink:
            model.addConstr(incoming == 1.0, name=f"sink_flow_{i}")
        else:
            model.addConstr(outgoing - incoming == 0.0, name=f"flow_{i}")

    model.optimize()

    if model.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Nominal problem not solved to optimality, status={model.Status}")

    adv_obj = model.ObjVal              # = min sum -p_i phi_ij ≤ 0
    total_duration = -adv_obj           # = max sum +p_i phi_ij ≥ 0

    phi_values = {(i, j): float(var.X) for (i, j), var in phi.items()}
    return total_duration, adv_obj, phi_values



In [5]:
def get_active_nodes_from_phi_simple(phi_values, n, source, sink, tol=1e-6):
    """
    n: number of nodes (nodes are 0,...,n-1)
    phi_values: dict {(i,j): value}
    returns list of nodes i (excluding source,sink) with sum_j phi_ij ≈ 1
    """
    outgoing_sum = [0.0] * n
    for (i, j), val in phi_values.items():
        outgoing_sum[i] += float(val)

    active_nodes = [
        i for i in range(n)
        if i not in (source, sink) and abs(outgoing_sum[i] - 1.0) <= tol
    ]
    return active_nodes

In [6]:
def build_worst_case_durations_simple(active_nodes, bar_p, hat_p, Gamma):
    """
    active_nodes: list of node indices on the robust path (excluding source,sink)
    bar_p, hat_p: lists
    Gamma: float

    p_i* = bar_p[i] + z_i * hat_p[i], with z_i ∈ {0, fractional, 1}.

    Returns:
        p_star: list of positive durations (worst-case realization).
    """
    n = len(bar_p)
    p_star = [float(bar_p[i]) for i in range(n)]

    # sort active nodes by hat_p[i] descending
    sorted_nodes = sorted(active_nodes, key=lambda i: float(hat_p[i]), reverse=True)

    floor_gamma = int(math.floor(Gamma))
    frac_gamma = float(Gamma - floor_gamma)

    for idx, i in enumerate(sorted_nodes, start=1):
        base = float(bar_p[i])
        hat = float(hat_p[i])

        if idx <= floor_gamma:
            z_i = 1.0
        elif idx == floor_gamma + 1 and frac_gamma > 0.0 and floor_gamma < len(sorted_nodes):
            z_i = frac_gamma
        else:
            z_i = 0.0

        p_star[i] = base + hat * z_i

    return p_star

In [7]:

def solve_adversary_over_all_pi_simple(bar_p, hat_p, edges, source, sink, Gamma, gurobi_output=False):
    """
    High-level adversary:

      For each pi in Pi:
        - Solve min sum -p_i(pi) phi_ij
        - Recover max project duration = -adv_obj

      Then:
        - Choose pi with minimal adv_obj (equivalently maximal project duration)
        - Extract active nodes and build p_star.

    Returns:
      best_pi
      best_duration        (positive project duration)
      best_adv_obj         (adversary objective, ≤ 0)
      best_phi
      active_nodes
      p_star               (positive worst-case durations)
      pi_values
    """
    n = len(bar_p)
    pi_values = build_pi_set(hat_p)

    best_pi = None
    best_adv_obj = math.inf      # we minimize the adversarial objective
    best_duration = None
    best_phi = None

    for pi in pi_values:
        total_duration, adv_obj, phi_values = solve_nominal_with_gurobi_simple(
            pi=pi,
            bar_p=bar_p,
            hat_p=hat_p,
            edges=edges,
            source=source,
            sink=sink,
            gurobi_output=gurobi_output,
        )

        # We want the "most damaging" scenario: minimal adv_obj (i.e., longest duration)
        if adv_obj < best_adv_obj:
            best_adv_obj = adv_obj
            best_duration = total_duration
            best_pi = pi
            best_phi = phi_values

    if best_phi is None:
        raise RuntimeError("No feasible pi found; check the input graph and durations.")

    active_nodes = get_active_nodes_from_phi_simple(
        phi_values=best_phi,
        n=n,
        source=source,
        sink=sink,
    )

    p_star = build_worst_case_durations_simple(
        active_nodes=active_nodes,
        bar_p=bar_p,
        hat_p=hat_p,
        Gamma=Gamma,
    )

    return best_pi, best_duration, best_adv_obj, best_phi, active_nodes, p_star, pi_values

**examples of diffrent inputs**

In [8]:
# ---------- Minimal example ----------

if __name__ == "__main__":
    # Nodes: 0 -> 1,2 -> 3
    bar_p = [0.0, 1.0, 0.5, 0.0]
    hat_p = [0.0, 0.8, 1.5, 0.0]

    edges = [(0, 1), (0, 2), (1, 3), (2, 3)]
    source = 0
    sink = 3
    Gamma = 1.5

    best_pi, best_duration, best_adv_obj, best_phi, active_nodes, p_star, pi_values = \
        solve_adversary_over_all_pi_simple(
            bar_p=bar_p,
            hat_p=hat_p,
            edges=edges,
            source=source,
            sink=sink,
            Gamma=Gamma,
            gurobi_output=True,
        )

    print("Pi values:", pi_values)
    print("Best pi:", best_pi)
    print("Best adversarial objective (min sum -p):", best_adv_obj)
    print("Best duration (max sum p):", best_duration)
    print("Active nodes:", active_nodes)
    print("Worst-case durations p*:", p_star)


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x11b801a4
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: -2 
No other solutions better than -2

Optimal solution found (tolerance 1.00e-04)
Best objective -2.000000000000e+00, best bound -2.000000000000e+00, 

In [9]:
import random
import math

def generate_random_inputs(n_nodes=8, edge_prob=0.3, gamma_fraction=0.5, seed=None):
    """
    Generate random test input for the adversary problem.
    Returns: bar_p, hat_p, edges, source, sink, Gamma

    - כל הקשתות היוצאות מאותו צומת i בעלות אותו משקל.
    - לצומת המקור (0) משקל 0 (גם bar וגם hat).
    - לצומת השקיעה (sink) גם אפשר לשים 0 אם רוצים.
    """
    if seed is not None:
        random.seed(seed)

    source = 0
    sink = n_nodes - 1

    # Nominal durations per node (positive)
    bar_p = [round(random.uniform(1.0, 5.0), 2) for _ in range(n_nodes)]
    hat_p = [round(random.uniform(0.0, 2.0), 2) for _ in range(n_nodes)]

    # Enforce: source (and optionally sink) have 0 duration
    bar_p[source] = 0.0
    hat_p[source] = 0.0
    bar_p[sink] = 0.0
    hat_p[sink] = 0.0

    # Always include a backbone path 0->1->2->...->n-1
    edges = [(i, i+1) for i in range(n_nodes - 1)]

    # Add random forward edges (i < j)
    for i in range(n_nodes):
        for j in range(i+2, n_nodes):
            if random.random() < edge_prob:
                edges.append((i, j))

    # Budget Γ proportional to number of internal nodes
    n_internal = max(0, n_nodes - 2)
    Gamma = round(gamma_fraction * n_internal, 2)

    return bar_p, hat_p, edges, source, sink, Gamma


In [10]:
# Example 1: small random graph
bar_p, hat_p, edges, source, sink, Gamma = generate_random_inputs(
    n_nodes=6, edge_prob=0.4, gamma_fraction=0.5, seed=42
)
print("bar_p =", bar_p)
print("hat_p =", hat_p)
print("edges =", edges)
print("Gamma =", Gamma)

bar_p_arc = {(i, j): bar_p[i] for (i, j) in edges}

hat_p_arc = {(i, j): hat_p[i] for (i, j) in edges}

print("bar_p_arc =", bar_p_arc)
print("hat_p_arc =", hat_p_arc)


bar_p = [0.0, 1.1, 2.1, 1.89, 3.95, 0.0]
hat_p = [0.0, 0.17, 0.84, 0.06, 0.44, 0.0]
edges = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (0, 2), (0, 3), (1, 3), (2, 4)]
Gamma = 2.0
bar_p_arc = {(0, 1): 0.0, (1, 2): 1.1, (2, 3): 2.1, (3, 4): 1.89, (4, 5): 3.95, (0, 2): 0.0, (0, 3): 0.0, (1, 3): 1.1, (2, 4): 2.1}
hat_p_arc = {(0, 1): 0.0, (1, 2): 0.17, (2, 3): 0.84, (3, 4): 0.06, (4, 5): 0.44, (0, 2): 0.0, (0, 3): 0.0, (1, 3): 0.17, (2, 4): 0.84}


In [11]:
result = solve_adversary_over_all_pi_simple(
    bar_p=bar_p,
    hat_p=hat_p,
    edges=edges,
    source=source,
    sink=sink,
    Gamma=Gamma,
    gurobi_output=False,
)
print(result)


(0.0, 10.55, -10.55, {(0, 1): 1.0, (1, 2): 1.0, (2, 3): 1.0, (3, 4): 1.0, (4, 5): 1.0, (0, 2): 0.0, (0, 3): 0.0, (1, 3): 0.0, (2, 4): 0.0}, [1, 2, 3, 4], [0.0, 1.1, 2.94, 1.89, 4.390000000000001, 0.0], [0.0, 0.06, 0.17, 0.44, 0.84])
